In [3]:
#This Code predict Label for SVF Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsRoslyn_20102019_Single.csv')
data = data.sample(frac=1)

y = list(data['Label']) 
x = list(data['Title']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

#80% / 20% train / test split:

train_size = int(len(x) * .8)

x_train = np.asarray(x_enc[:train_size])
y_train = np.asarray(y_enc[:train_size])

x_test = np.asarray(x_enc[train_size:])
y_test = np.asarray(y_enc[train_size:])

Using TensorFlow backend.


In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [6]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=60, batch_size=32)
    model.save_weights('./Roslyn_T_Label_Predict_Single.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/60



715/715 [==============================] - 11s 15ms/step - loss: 2.2821 - acc: 0.5105
Epoch 2/60
715/715 [==============================] - 9s 12ms/step - loss: 1.2719 - acc: 0.6224
Epoch 3/60
715/715 [==============================] - 9s 12ms/step - loss: 1.1411 - acc: 0.6182
Epoch 4/60
715/715 [==============================] - 9s 12ms/step - loss: 1.0819 - acc: 0.6503
Epoch 5/60
715/715 [==============================] - 9s 12ms/step - loss: 1.0311 - acc: 0.6881
Epoch 6/60
715/715 [==============================] - 9s 12ms/step - loss: 0.9981 - acc: 0.6951
Epoch 7/60
715/715 [==============================] - 9s 12ms/step - loss: 0.9663 - acc: 0.6979
Epoch 8/60
715/715 [==============================] - 8s 12ms/step - loss: 0.9418 - acc: 0.7063
Epoch 9/60
715/715 [==============================] - 9s 12ms/step - loss: 0.9135 - acc: 0.7063
Epoch 10/60
715/715 [==============================] - 8s 12ms/step - loss: 0.9029 - acc: 0.7133
Epoch 11/60
715/715 [============================

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./Roslyn_T_Label_Predict_Single.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [9]:
y_test = np.asarray(y_enc[train_size:])
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))



                             precision    recall  f1-score   support

              4 - In Review       0.00      0.00      0.00         3
             Area-Analyzers       0.00      0.00      0.00         5
             Area-Compilers       0.48      0.65      0.55        40
              Area-External       0.00      0.00      0.00         1
                   Area-IDE       0.72      0.74      0.73        85
        Area-Infrastructure       0.86      0.83      0.85        36
           Area-Interactive       0.00      0.00      0.00         2
       Area-Language Design       0.00      0.00      0.00         1
PR For Personal Review Only       0.00      0.00      0.00         2
       Resolution-By Design       0.00      0.00      0.00         1
       Resolution-Duplicate       0.00      0.00      0.00         1
                        nan       0.00      0.00      0.00         2

                   accuracy                           0.66       179
                  macro avg     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# New Section